# Light curve of PKS 2155-304

- Data from the HESS DL3 DR1
- PKS 2155-304 flare during the night from July 29 to 30 2006
- See: https://docs.gammapy.org/0.17/notebooks/light_curve_flare.html

Divided into 3 notebooks
- data reduction
- standard fitting of flux points - Using `LightCurveEstimator`
- using temporal model - By joint fitting

## Setup 


In [1]:
%matplotlib inline
import astropy.units as u
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.time import Time
from regions import CircleSkyRegion
from astropy.coordinates import Angle

import logging

log = logging.getLogger(__name__)

Now let's import gammapy specific classes and functions

In [2]:
from gammapy.data import DataStore
from gammapy.datasets import SpectrumDataset, Datasets
from gammapy.modeling.models import PowerLawSpectralModel, SkyModel
from gammapy.maps import MapAxis
from gammapy.estimators import LightCurveEstimator
from gammapy.makers import (
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
)

## Select the data

We first set the datastore.

In [3]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1/")

Now we select observations within 2 degrees of PKS 2155-304. 

In [4]:
target_position = SkyCoord(
    329.71693826 * u.deg, -30.2255890 * u.deg, frame="icrs"
)
selection = dict(
    type="sky_circle",
    frame="icrs",
    lon=target_position.ra,
    lat=target_position.dec,
    radius=2 * u.deg,
)
obs_ids = data_store.obs_table.select_observations(selection)["OBS_ID"]
observations = data_store.get_observations(obs_ids)
print(f"Number of selected observations : {len(observations)}")

Number of selected observations : 21


## Define time intervals
We create the list of time intervals. Each time interval is an `~astropy.time.Time` object, containing a start and stop time.

In [5]:
t0 = Time("2006-07-29T20:30")
t_max = t0 + 12*u.hr
duration = 10.0 * u.min
n_time_bins = 12*u.hr / duration
times = t0 + np.arange(n_time_bins) * duration
time_intervals = [
    Time([tstart, tstop]) for tstart, tstop in zip(times[:-1], times[1:])
]
print(len(time_intervals))

71


## Filter the observations list in time intervals

Here we apply the list of time intervals to the observations with `~gammapy.data.Observations.select_time()`.

This will return a new list of Observations filtered by time_intervals. For each time interval, a new observation is created that convers the intersection of the GTIs and time interval. 

In [6]:
short_observations = observations.select_time(time_intervals)
# check that observations have been filtered
print(
    f"Number of observations after time filtering: {len(short_observations)}\n"
)
print(len(short_observations))

Number of observations after time filtering: 59

59


In [7]:
print(short_observations[0].gti.time_delta)

[600.] s


## Building 1D datasets from the new observations

Here we will perform the data reduction in 1D with reflected regions.

**Beware, with small time intervals the background normalization with OFF regions might become problematic.**

### Defining the geometry

We define the energy axes. As usual, the true energy axis has to cover a wider range to ensure a good coverage of the measured energy range chosen. 

We need to define the ON extraction region. Its size follows typical spectral extraction regions for HESS analyses.

In [8]:
# Target definition
e_reco = MapAxis.from_energy_bounds(0.4, 20, 10, "TeV").edges
e_true = MapAxis.from_energy_bounds(0.1, 40, 20, "TeV").edges

on_region_radius = Angle("0.11 deg")
on_region = CircleSkyRegion(center=target_position, radius=on_region_radius)

### Creation of the data reduction makers

We now create the dataset and background makers for the selected geometry.

In [9]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=True, selection=["counts", "aeff", "edisp"]
)
bkg_maker = ReflectedRegionsBackgroundMaker()
safe_mask_masker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

### Creation of the datasets

Now we perform the actual data reduction in the time_intervals.

In [10]:
%%time
datasets = []

dataset_empty = SpectrumDataset.create(
    e_reco=e_reco, e_true=e_true, region=on_region
)

for obs in short_observations:
    dataset = dataset_maker.run(dataset_empty.copy(), obs)

    dataset_on_off = bkg_maker.run(dataset, obs)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, obs)
    datasets.append(dataset_on_off)

CPU times: user 14.6 s, sys: 495 ms, total: 15.1 s
Wall time: 15.2 s


In [11]:
# Save these short time datasets:
Datasets(datasets).write("./", overwrite=True)